In [5]:
from pydap.client import open_url
from datetime import datetime
import numpy as np
import pandas as pd
import time
from datetime import timedelta
import pyproj
import xarray as xr
from scipy.interpolate import LinearNDInterpolator
import matplotlib.pyplot as plt
import math
import pickle
from datetime import date
import os
import plotly.express as px
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import matplotlib
import cartopy as cart

In [29]:
def hours_since_ref(year, month, day):
    d0 = date(1992, 10, 5)
    d1 = date(year, month, day)
    delta = d1 - d0
    hours = delta.days*24
    return hours

def get_cygnss_data(year, month, day):
    cygnss_df = pd.DataFrame()
    hours = hours_since_ref(year, month, day)
    test_data_url, test_clickable_url = generate_url(year, month, day)
    
    not_got_data = True
    tries = 0
    while not_got_data:
        try: 
            dataset = open_url(test_data_url, output_grid=False)
            not_got_data = False
        except:
            if tries < 50:
                tries += 1
            else:
                return pd.DataFrame()

    
    df = pd.DataFrame()
    
    mss = np.array(dataset.mean_square_slope[:])
    lat = np.array(dataset.lat[:])
    lon = np.array(dataset.lon[:])
    sample_time = np.array(dataset.sample_time[:])
    sample_time = np.array(sample_time / 3600) + hours

    df['mss'] = mss.tolist()
    df['lat'] = lat.tolist()
    df['lon'] = lon.tolist()
    df['sample_time'] = sample_time.tolist()
    return df

def prep_cygnss(cygnss_df):
    np.warnings.filterwarnings('ignore')
    for key in cygnss_df:
        cygnss_df = cygnss_df[cygnss_df[key] != -9999.0]
    cygnss_df.dropna(inplace = True)
    return cygnss_df

def generate_url(year, month, day):

    day_of_year = datetime(year, month, day).timetuple().tm_yday
    date_string = str(year) + str(month).zfill(2) + str(day).zfill(2)

    base_url = 'https://podaac-opendap.jpl.nasa.gov/opendap/hyrax/allData/cygnss/L2/v2.1/'
    specific_url = str(year) + '/' + str(day_of_year).zfill(3) + '/cyg.ddmi.s' + \
                   date_string + '-000000-e' + date_string + '-235959.l2.wind-mss.a21.d21.nc'
    data_url = base_url + specific_url
    clickable_url = base_url + specific_url + '.html'

    return data_url + '?lat,lon,mean_square_slope,sample_time', clickable_url

def fetch_cygnss(y1, m1, d1, y2, m2, d2):
    sdate = date(y1, m1, d1)   # start date
    edate = date(y2, m2, d2)   # end date
    delta = edate - sdate       # as timedelta
    df_list = []
    for i in range(delta.days + 1):
        print("Proceccing day " + str(i + 1) + " of total : " + str(delta.days +1) + " days")
        day = sdate + timedelta(days=i)
        df = get_cygnss_data(day.year, day.month, day.day)
        if not df.empty:
            df = prep_cygnss(df)
            df_list.append(df)
            df.to_csv("level_2_mss/" + str(day.year) + str(day.month) + str(day.day) + ".csv" ,index=False)
    if len(df_list) == 1:
        return df_list[0]
    else:
        return pd.concat(df_list)

In [30]:
cygnss_df = fetch_cygnss(2021, 11, 7, 2021, 11, 8)

Proceccing day 1 of total : 2 days
Proceccing day 2 of total : 2 days


In [31]:
cygnss_df

,mss,lat,lon,sample_time
0,0.040190,27.068745,238.914627,255000.000000
4,0.030616,-13.275762,80.261353,255000.000000
6,0.074395,-28.711931,339.701050,255000.000000
9,0.039546,-30.074169,352.365875,255000.000000
10,0.084171,5.394580,87.261581,255000.000000
...,...,...,...,...
1372571,0.041299,11.627394,95.446220,255047.999757
1372573,0.035981,17.176819,93.450012,255047.999757
1372575,0.056649,6.821241,82.376236,255047.999861
1372576,0.024015,13.051071,97.901848,255047.999861


In [136]:
def fetch_oscar_data(start_day, end_day):
    for i in range(start_day, end_day+1):
        res = fetch_oscar_one_day(i)
        if res:
            u_list = []
            v_list = []
            lat_list = []
            lon_list = []
            for index, lat in enumerate(res[0]):
                for index2, lon in enumerate(res[1]):
                    u_list.append(res[2][index][index2])
                    v_list.append(res[3][index][index2])
                    lat_list.append(lat)
                    lon_list.append(lon)
            df = pd.DataFrame()
            df['u'] = u_list
            df['v'] = v_list
            df['lat'] = lat_list
            df['lon'] = lon_list
            df['time'] = np.zeros(len(df['lon']))
            df = df.assign(time=i*24)
            df = df.loc[df['lon'] < 380 ]
            df['lon'] = df['lon']%360
            df.to_csv("oskar_data/" + str(i) + ".csv" ,index=False)
        
def fetch_oscar_one_day(day):
    try:
        url = 'https://podaac-opendap.jpl.nasa.gov/opendap/allData/oscar/preview/L4/oscar_third_deg/oscar_vel' + str(day) + '.nc.gz?latitude[0:1:480],longitude[0:1:1200],u[0:1:0][0:1:0][0:1:480][0:1:1200],v[0:1:0][0:1:0][0:1:480][0:1:1200]'       
        dataset = open_url(url, output_grid=False)
        lats = np.array(dataset.latitude[:])
        lons = np.array(dataset.longitude[:])
        u = np.squeeze(np.array(dataset.u[:][:][:][:]))            
        v = np.squeeze(np.array(dataset.v[:][:][:][:]))         
        print("fetched")
        return lats, lons, u, v
    except:
        print("Not a date")  
        return None


In [137]:
res = fetch_oscar_data(10004, 10004)

fetched


AttributeError: 'NoneType' object has no attribute 'dropna'

In [134]:
max(res['lon'])

359.6666666666667

,u,v,lat,lon
62,0.011389,-0.013964,80.000000,40.666667
63,0.009841,-0.014871,80.000000,41.000000
64,0.008030,-0.019606,80.000000,41.333333
65,0.006164,-0.024512,80.000000,41.666667
66,0.004116,-0.018465,80.000000,42.000000
...,...,...,...,...
569803,-0.034375,-0.014704,-78.000000,196.333333
569804,-0.027573,-0.019918,-78.000000,196.666667
569805,-0.022020,-0.002575,-78.000000,197.000000
569806,-0.018097,0.010140,-78.000000,197.333333


In [90]:
def open_oskar_data_local(filename):
    ds = xr.open_dataset(filename)
    oskar_df = ds.to_dataframe()
    oskar_df.dropna(inplace = True)
    oskar_df = oskar_df.reset_index()
    d0 = date(1992, 10, 5)
    d1 = oskar_df['time'][1].date()
    delta = d1 - d0
    hours = delta.days*24
    oskar_df['time'] = np.zeros(len(oskar_df['time']))
    oskar_df = oskar_df.assign(time=hours)
    oskar_df = oskar_df.rename(columns={"latitude": "sp_lat", "longitude": "sp_lon", "time" : "hours_since_ref"})
    oskar_df = oskar_df.loc[oskar_df['sp_lon'] < 380 ]
    oskar_df['sp_lon'] = oskar_df['sp_lon']%360
    oskar_df = reduce_area_of_df(oskar_df)
    return oskar_df

In [99]:
lats, lons, u, v, tid

NameError: name 'lats' is not defined